In [12]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [13]:
# Set the tracking URI to the local mlruns directory
mlflow.set_tracking_uri("file:///Users/mustainbillah/Projects/sentiment-analysis/mlruns")

# Create and set the experiment
experiment_name = "04_ML_Algorithm_Comparison_with_HPT"
try:
    experiment_id = mlflow.create_experiment(experiment_name)
except:
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///Users/mustainbillah/Projects/sentiment-analysis/mlruns/658432777505770578', creation_time=1748791728665, experiment_id='658432777505770578', last_update_time=1748791728665, lifecycle_stage='active', name='04_ML_Algorithm_Comparison_with_HPT', tags={}>

In [14]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [15]:
import mlflow.sklearn


df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

df = df.dropna(subset=['category'])

ngram_range = (1, 3)
max_features = 1000

X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run() as run:
        mlflow.set_tag('mlflow.runName', f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag('experiment_type', 'algorithm_comparison')

        mlflow.log_param('algorithm_name', model_name)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")

def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42
    )
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


def run_optuna_experiment():
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_xgboost, n_trials=30)

    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'],
                              learning_rate=best_params['learning_rate'],
                              max_depth=best_params['max_depth'],
                              random_state=42)
    
    log_mlflow('XGBoost', best_model, X_train_vec, X_test_vec, y_train, y_test)

run_optuna_experiment()
    

    




[I 2025-06-01 21:28:54,739] A new study created in memory with name: no-name-f9dadd73-2e06-4e99-ad91-a180f5fc71a6
[I 2025-06-01 21:29:12,296] Trial 0 finished with value: 0.5284331105959362 and parameters: {'n_estimators': 298, 'learning_rate': 0.00010475374250104919, 'max_depth': 4}. Best is trial 0 with value: 0.5284331105959362.
[I 2025-06-01 21:29:24,408] Trial 1 finished with value: 0.5547524887494886 and parameters: {'n_estimators': 87, 'learning_rate': 0.0005114273510169308, 'max_depth': 6}. Best is trial 1 with value: 0.5547524887494886.
[I 2025-06-01 21:29:36,867] Trial 2 finished with value: 0.6881221873721532 and parameters: {'n_estimators': 95, 'learning_rate': 0.047860540179507784, 'max_depth': 6}. Best is trial 2 with value: 0.6881221873721532.
[I 2025-06-01 21:30:18,645] Trial 3 finished with value: 0.591981453702441 and parameters: {'n_estimators': 182, 'learning_rate': 0.00021989445503352252, 'max_depth': 8}. Best is trial 2 with value: 0.6881221873721532.
[I 2025-06-0